## Install dependencies

In [1]:
!python3 -m pip install -r requirements.txt

  Using cached kubeflow-metadata-0.3.1.tar.gz (10 kB)
ERROR: Could not find a version that satisfies the requirement ml-metadata==0.24.0 (from versions: 0.12.0.dev0, 0.13.0.dev0, 0.13.1.dev0)
ERROR: No matching distribution found for ml-metadata==0.24.0
You should consider upgrading via the '/Users/josafat/1-Documents/4_Studium/IT-Systems-Engineering/2_Skripte/2-Master/2. Semester/Building Machine Learning Applications/BirdSongIdentification/venv/bin/python3 -m pip install --upgrade pip' command.


## Setup file path manager & logging


In [1]:
import logging

from data_preparation import filepaths
from general.logging import logger

path_manager = filepaths.PathManager("./data")

logger.setLevel(logging.VERBOSE)

## Step 1: Download audio files and metadata from xeno-canto

In [2]:
from data_preparation import downloader

downloader = downloader.XenoCantoDownloader(path_manager)

downloader.create_datasets(
    species_list=["Petronia petronia, call", "Petronia petronia, song"],
    use_nips4b_species_list=False,
    maximum_samples_per_class=10,
    test_size=0.35,
    min_quality="E",
    sound_types=["song", "call"],   
    sexes=None,
    life_stages=None,
    exclude_special_cases=True,
    maximum_number_of_background_species=1,
    clear_audio_cache=False,
    clear_label_cache=False,
    )

Download label file for Petronia petronia...: 0it [00:00, ?it/s]

sound types ['song', 'call']
selected_sound_types {'song', 'call'}
Sound types for Petronia petronia: {'song', 'call'}
empty dir ./data/train/audio
empty dir ./data/val/audio
empty dir ./data/test/audio


Download training set...:   0%|          | 0/6 [00:00<?, ?it/s]

download https://www.xeno-canto.org/464984/download
download https://www.xeno-canto.org/567431/download
download https://www.xeno-canto.org/369008/download
download https://www.xeno-canto.org/128141/download
download https://www.xeno-canto.org/279048/download
Could not download file with id 3      567431
116    370012
106    464984
117    369008
71     128141
180    279048
Name: id, dtype: object


Download validation set...:   0%|          | 0/2 [00:00<?, ?it/s]

download https://www.xeno-canto.org/498710/download
download https://www.xeno-canto.org/460339/download


Download test set...:   0%|          | 0/2 [00:00<?, ?it/s]

download https://www.xeno-canto.org/414373/download
download https://www.xeno-canto.org/567432/download



## Step 2: Create spectrograms from audio files

In [3]:
from data_preparation import spectrograms

spectrogram_creator = spectrograms.SpectrogramCreator(
            1000,
            path_manager,
            include_noise_samples=True)

spectrogram_creator.create_spectrograms_for_splits(splits=["train", "val", "test"], clear_spectrogram_cache=False)

empty dir ./data/train/spectrograms_1000


Create spectrograms for training set:   0%|          | 0/5 [00:00<?, ?it/s]

empty dir ./data/val/spectrograms_1000


Create spectrograms for validation set:   0%|          | 0/2 [00:00<?, ?it/s]

empty dir ./data/test/spectrograms_1000


Create spectrograms for test set:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 3: Train model

In [4]:
from training import training

trainer = training.ModelTrainer(
    path_manager,
    architecture="resnet18",
    chunk_length=1000,
    experiment_name="Test run",
    batch_size=100,
    include_noise_samples=True,
    layers_to_unfreeze=["layer3", "layer4", "avg_pool", "fc"],
    learning_rate=0.001,
    learning_rate_scheduler=None,
    learning_rate_scheduler_gamma=0.1,
    learning_rate_scheduler_step_size=7,
    multi_label_classification=True,
    multi_label_classification_threshold=0.5,
    number_epochs=1,
    number_workers=0,
    optimizer="Adam",
    track_metrics=False,
    wandb_entity_name="",
    wandb_key="",
    wandb_project_name=""
)
best_average_model, best_minimum_model, best_models_per_class = trainer.train_model()

Number of species: 3
Epoch 1/1
----------

| metric          |  Petronia_petronia_call  |  Petronia_petronia_song  |  noise  |
|-----------------|--------------------------|--------------------------|---------|
| f1-score        |          0.5479          |          0.2027          | 0.6242  |
| precision       |          0.5063          |          0.1128          | 0.9423  |
| recall          |          0.5970          |          1.0000          | 0.4667  |
|                 |                          |                          |         |
| true-positives  |         40.0000          |         15.0000          | 49.0000 |
| true-negatives  |         81.0000          |         54.0000          | 79.0000 |
| false-positives |         39.0000          |         118.0000         | 3.0000  |
| false-negatives |         27.0000          |          0.0000          | 56.0000 |


| metric          |  Petronia_petronia_call  |  Petronia_petronia_song  |  noise  |
|-----------------|------------

## Step 4: Evaluate model

In [5]:
from training import model_evaluator

evaluator = model_evaluator.ModelEvaluator(path_manager,
                                            architecture="resnet18",
                                            batch_size=100,
                                            chunk_length=1000,
                                            include_noise_samples=True,
                                            multi_label_classification=True,
                                            track_metrics=False,
                                            test="test")

evaluator.evaluate_model_on_test_set(model=best_average_model, model_name="test_model",)

Model performance of test_model on evaluation set:

| metric          |  Petronia_petronia_call  |  Petronia_petronia_song  |  noise  |
|-----------------|--------------------------|--------------------------|---------|
| f1-score        |          0.0000          |          0.9683          | 0.0000  |
| precision       |          0.0000          |          0.9385          | 0.0000  |
| recall          |          0.0000          |          1.0000          | 0.0000  |
|                 |                          |                          |         |
| true-positives  |          0.0000          |         61.0000          | 0.0000  |
| true-negatives  |          2.0000          |          0.0000          | 61.0000 |
| false-positives |         63.0000          |          4.0000          | 0.0000  |
| false-negatives |          0.0000          |          0.0000          | 4.0000  |

